In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = 'gdrive/My Drive/data.csv'

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv(path)

In [0]:
data = data[['category_name', 'title']]

In [95]:
data.head()

,category_name,title
0,Детская одежда и обувь,Платье праздничное для девочки
1,Мебель и интерьер,Кровать двухспальная
2,Телефоны,Samsung S3 mini gt-i8190
3,"Одежда, обувь, аксессуары","Красивое платье на одно плечо, в идеале размер м"
4,Детская одежда и обувь,Стильная панама


In [0]:
pd.set_option('display.max_rows', 500)

In [0]:
from sklearn.cluster import AffinityPropagation, AgglomerativeClustering, DBSCAN, \
                            KMeans, MiniBatchKMeans, Birch, MeanShift, SpectralClustering
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, \
                            silhouette_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn import metrics
from sklearn.datasets import make_blobs

#Задание 1

AffinityPropagation

In [0]:
sample = data.sample(frac=0.01)

In [0]:
cv = CountVectorizer(max_features=500, min_df=4, max_df=0.3, ngram_range = (1,3))
tf = TfidfVectorizer(max_features = 500, min_df = 4, max_df = 0.3, ngram_range = (1,3))

X_cv = cv.fit_transform(sample['title'])
X_tf = tf.fit_transform(sample['title'])

y = sample['category_name']

In [108]:
AP_cluster_cv = AffinityPropagation(damping=0.8, preference=-2, 
                              max_iter=400, verbose=2)
AP_cluster_cv.fit(X_cv)

Converged after 349 iterations.


AffinityPropagation(affinity='euclidean', convergence_iter=15, copy=True,
                    damping=0.8, max_iter=400, preference=-2, verbose=2)

In [109]:
AP_cluster_tf = AffinityPropagation(damping=0.8, preference=-2, 
                              max_iter=400, verbose=2)
AP_cluster_tf.fit(X_tf)

Converged after 336 iterations.


AffinityPropagation(affinity='euclidean', convergence_iter=15, copy=True,
                    damping=0.8, max_iter=400, preference=-2, verbose=2)

In [0]:
AP_labels_cv = AP_cluster_cv.labels_
AP_labels_tf = AP_cluster_tf.labels_

In [0]:
def silh(X, labels):
  print("Silhouette Coefficient: %0.3f"
        % metrics.silhouette_score(X[:10000], labels[:10000]))

In [112]:
silh(X_cv, AP_labels_cv)

Silhouette Coefficient: 0.562


In [113]:
silh(X_tf, AP_labels_tf)

Silhouette Coefficient: 0.566


In [0]:
def score(y, labels):
  # эти три метрки что-то вроде точности, полноты и ф-меры для кластеризации
  print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
  print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
  print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

  print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(y, labels))
  print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(y, labels))

In [115]:
score(y, AP_labels_cv)

Homogeneity: 0.578
Completeness: 0.376
V-measure: 0.456
Adjusted Rand Index: -0.006
Adjusted Mutual Information: 0.223


In [116]:
score(y, AP_labels_tf)

Homogeneity: 0.559
Completeness: 0.389
V-measure: 0.459
Adjusted Rand Index: -0.003
Adjusted Mutual Information: 0.277


Здесь у tfidf показатели получше, хотя cv выигрывает в гомогенности (незначительно). 

Kmeans

In [0]:
sample = data.sample(frac=0.1)

In [0]:
X_cv = cv.fit_transform(sample['title'])
X_tf = tf.fit_transform(sample['title'])

y = sample['category_name']

In [121]:
KM_cluster_cv = KMeans(n_clusters=100, n_init=20, tol = 0.00001)
KM_cluster_cv.fit(X_cv)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=100, n_init=20, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=1e-05, verbose=0)

In [122]:
KM_cluster_tf = KMeans(n_clusters=100, n_init=12, tol = 0.00001)
KM_cluster_tf.fit(X_tf)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=100, n_init=12, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=1e-05, verbose=0)

In [0]:
KM_labels_cv = KM_cluster_cv.labels_

In [0]:
KM_labels_tf = KM_cluster_tf.labels_

In [125]:
silh(X_cv, KM_labels_cv)

Silhouette Coefficient: 0.331


In [126]:
silh(X_tf, KM_labels_tf)

Silhouette Coefficient: 0.396


In [127]:
score(y, KM_labels_cv)

Homogeneity: 0.315
Completeness: 0.350
V-measure: 0.331
Adjusted Rand Index: -0.021
Adjusted Mutual Information: 0.312


In [128]:
score(y, KM_labels_tf)

Homogeneity: 0.366
Completeness: 0.361
V-measure: 0.364
Adjusted Rand Index: -0.008
Adjusted Mutual Information: 0.345


Классификатор KMeans обученный при помощи tfidf-векторизатора всё равно показывает лучший результат, чем классификатор, обученный при помощи countvectorizer'а. Однако первый классификатор оказался более чувствителен к увеличению количества инициализаций (n_init). Качество, правда, росло совсем незначительно. 

In [0]:
sample['cluster'] = KM_cluster_tf.labels_

MiniBatchKMeans

In [0]:
X_cv = cv.fit_transform(data['title'])
X_tf = tf.fit_transform(data['title'])

y = data['category_name']

In [0]:
MBK_cluster_cv = MiniBatchKMeans(n_clusters=1000, init_size=2000, verbose=1, max_iter=500, batch_size = 200,
                          max_no_improvement=2000, reassignment_ratio=0.4)
MBK_cluster_cv.fit(X_cv)

In [0]:
MBK_cluster_tf = MiniBatchKMeans(n_clusters=1000, init_size=2000, verbose=1, max_iter=500, batch_size = 200,
                          max_no_improvement=600, reassignment_ratio=0.5)
MBK_cluster_tf.fit(X_tf)

In [0]:
MBK_labels_cv = MBK_cluster_cv.labels_

In [0]:
MBK_labels_tf = MBK_cluster_tf.labels_

In [136]:
silh(X_cv, MBK_labels_cv)

Silhouette Coefficient: 0.639


In [137]:
silh(X_tf, MBK_labels_tf)

Silhouette Coefficient: 0.689


In [138]:
score(y, MBK_labels_cv)

Homogeneity: 0.516
Completeness: 0.306
V-measure: 0.384
Adjusted Rand Index: -0.012
Adjusted Mutual Information: 0.372


In [139]:
score(y, MBK_labels_tf)

Homogeneity: 0.520
Completeness: 0.308
V-measure: 0.387
Adjusted Rand Index: -0.013
Adjusted Mutual Information: 0.375


Лучше справляется tfidf-векторизатор. CV тоже можно подкрутить, но у меня он всё равно так и не достиг результатов tfidf. Особенно заметна разница с MiniBatchKMeans.

#Задание 2

AgglomerativeClustering

In [0]:
sample = data.sample(frac=0.05)

In [0]:
svd = TruncatedSVD(50)
nmf = NMF(init='random', random_state=0)

In [0]:
X_tf = tf.fit_transform(sample['title'])

In [0]:
X_svd = svd.fit_transform(X_tf)
X_nmf = nmf.fit_transform(X_tf)

In [0]:
y = sample['category_name']

In [145]:
AC_cluster_svd = AgglomerativeClustering(n_clusters=200)
AC_cluster_svd.fit(X_svd)

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='ward', memory=None, n_clusters=200)

In [146]:
AC_cluster_nmf = AgglomerativeClustering(n_clusters=200)
AC_cluster_nmf.fit(X_nmf)

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='ward', memory=None, n_clusters=200)

In [0]:
AC_labels_svd = AC_cluster_svd.labels_

In [0]:
AC_labels_nmf = AC_cluster_nmf.labels_

In [149]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X_svd, AC_labels_svd))

Silhouette Coefficient: 0.630


In [150]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X_nmf, AC_labels_nmf))

Silhouette Coefficient: 0.434


In [153]:
score(y, AC_labels_svd)

Homogeneity: 0.414
Completeness: 0.345
V-measure: 0.376
Adjusted Rand Index: -0.006
Adjusted Mutual Information: 0.325


In [154]:
score(y, AC_labels_nmf)

Homogeneity: 0.388
Completeness: 0.329
V-measure: 0.356
Adjusted Rand Index: -0.020
Adjusted Mutual Information: 0.300



Я пробовал менять показатели affinity и linkage для AC, однако со стандартными параметрами (немного подкрутил n_clusters) работала она лучше. 

Spectral Clustering



In [0]:
sample = data.sample(frac=0.01)

In [0]:
X_tf = tf.fit_transform(sample['title'])

In [0]:
X_svd = svd.fit_transform(X_tf)
X_nmf = nmf.fit_transform(X_tf)

In [0]:
y = sample['category_name']

In [166]:
SC_cluster_svd = SpectralClustering(n_clusters=100, n_init = 5, gamma=1.2)
SC_cluster_svd.fit(X_svd)

SpectralClustering(affinity='rbf', assign_labels='kmeans', coef0=1, degree=3,
                   eigen_solver=None, eigen_tol=0.0, gamma=1.2,
                   kernel_params=None, n_clusters=100, n_components=None,
                   n_init=5, n_jobs=None, n_neighbors=10, random_state=None)

In [167]:
SC_cluster_nmf = SpectralClustering(n_clusters=100, n_init=5, gamma=1.2)
SC_cluster_nmf.fit(X_nmf)

SpectralClustering(affinity='rbf', assign_labels='kmeans', coef0=1, degree=3,
                   eigen_solver=None, eigen_tol=0.0, gamma=1.2,
                   kernel_params=None, n_clusters=100, n_components=None,
                   n_init=5, n_jobs=None, n_neighbors=10, random_state=None)

In [0]:
SC_labels_svd = SC_cluster_svd.labels_

In [0]:
SC_labels_nmf = SC_cluster_nmf.labels_

In [170]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X_svd, SC_labels_svd))

Silhouette Coefficient: 0.600


In [171]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X_nmf, SC_labels_nmf))

Silhouette Coefficient: 0.332


In [172]:
score(y, SC_labels_svd)

Homogeneity: 0.394
Completeness: 0.388
V-measure: 0.391
Adjusted Rand Index: -0.010
Adjusted Mutual Information: 0.280


In [173]:
score(y, SC_labels_nmf)

Homogeneity: 0.119
Completeness: 0.382
V-measure: 0.182
Adjusted Rand Index: 0.006
Adjusted Mutual Information: 0.075


svd пока лучше по всем показателям

MeanShift

In [0]:
sample = data.sample(frac=0.01)

In [0]:
X_tf = tf.fit_transform(sample['title'])

In [0]:
X_svd = svd.fit_transform(X_tf)
X_nmf = nmf.fit_transform(X_tf)

In [0]:
y = sample['category_name']

In [183]:
import sklearn
sklearn.cluster.estimate_bandwidth(X_svd)

0.4645504738146529

In [184]:
MS_cluster_svd = MeanShift(cluster_all=False, bandwidth=0.7, max_iter = 400)
MS_cluster_svd.fit(X_svd)

MeanShift(bandwidth=0.7, bin_seeding=False, cluster_all=False, max_iter=400,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [185]:
MS_cluster_nmf = MeanShift(cluster_all=False, bandwidth=0.7, max_iter = 400)
MS_cluster_nmf.fit(X_nmf)

MeanShift(bandwidth=0.7, bin_seeding=False, cluster_all=False, max_iter=400,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [0]:
MS_labels_svd = MS_cluster_svd.labels_

In [0]:
MS_labels_nmf = MS_cluster_nmf.labels_

In [188]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X_svd, MS_labels_svd))

Silhouette Coefficient: 0.461


In [189]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X_nmf, MS_labels_nmf))

Silhouette Coefficient: 0.607


In [190]:
score(y, MS_labels_svd)

Homogeneity: 0.130
Completeness: 0.282
V-measure: 0.178
Adjusted Rand Index: -0.028
Adjusted Mutual Information: 0.125


In [191]:
score(y, MS_labels_nmf)

Homogeneity: 0.572
Completeness: 0.368
V-measure: 0.447
Adjusted Rand Index: -0.021
Adjusted Mutual Information: 0.161


А вот здесь уже nmf значительно лучше.

На 2 из 3 моделей svd показало результаты лучше, чем nmf, однако c MeanShift nmf оказалось значительно лучше. В общем, получается, что svd получше, но явно не во всех случаях. 

#Задание 3

In [0]:
sample['cluster'] = MS_cluster_svd.labels_

In [196]:
sample[sample.cluster==-1].head(50)

,category_name,title,cluster
6314,Детская одежда и обувь,Костюм для карате,-1
109546,Коллекционирование,На концертах Владимира Высоцкого,-1
30513,Детская одежда и обувь,Комбинезон весна-осень,-1
8107,Мебель и интерьер,Светильники для спальни,-1
85141,"Одежда, обувь, аксессуары",Брючный костюм с жилетом,-1
160052,Детская одежда и обувь,Сапоги резиновые,-1
153541,Квартиры,"1-к квартира, 36 м², 4/10 эт.",-1
247091,Товары для детей и игрушки,Балдахин и держатель для него,-1
81491,"Одежда, обувь, аксессуары",Продам женскуюодежду,-1
78398,Товары для компьютера,"Новая топовая сво ekwb, полный комплект",-1


Для некоторых заголовков вполне можно объяснить, почему они попали в мусорный класс. "Кофе на вынос", например, действительно сложно было бы отнести к категории "Готовый бизнес" (нет никаких значимых слов, говорящих о принадлежности к классу, и, скорее всего, подобных предложений больше не встречалось). Встречаются варианты типа "Продам женскуюодежду", что также определяется в мусорный класс (наверное, алгоритмы кластеризации не умеют работать с такими вот слитно написанными словами). Встречалась и просто редкая лексика, типа "кожанка". Учитывая, что контекс зачастую ничего не говорит о принадлежности к классу ("Продам..." или "Куплю...") - ничего удивительного. Но странно, что заголовки типа  "Продам книги: Фантастический детектив" не попали в отдельный кластер. Возможно, те заголовки, которые по каким-то причинам попали в мусорный кластер, обладая при этом явной принадлежностью к чему-то, не добрали по количеству и не смогли образовать собственный кластер. 

In [198]:
sample[sample.category_name=='Книги и журналы'].cluster.value_counts()

 0     10
-1      2
 16     1
Name: cluster, dtype: int64

Видимо, да. Слишком мало вхождений.

In [199]:
sample[sample.category_name=='Готовый бизнес'].cluster.value_counts()

 0    3
-1    3
Name: cluster, dtype: int64

И здесь тоже

В общем, основная причина попадания в мусорный класс - судя по всему, недобор. Слишком мало вхождений на категорию, а значит алгоритм не может собрать кластер и считает такие штуки выбросами. 